In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-buior6cd
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-buior6cd
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 5741c522547756ac4bb7a16df32106a15efb8a57
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10742 sha256=8f76f24c8ef76f4567737825c96eef1bce4cf6e61dca70f1aea8c5c7f7a278f6
  Stored in directory: /tmp/pip-ephem-wheel-cache-xp2lf6df/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [ ]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpgcs4ecg8".


In [ ]:
%%cuda
#include <cstdio>
#include <iostream>

	using namespace std;

__global__ void maxi(int* a, int* b, int n)
{
	int block = 256 * blockIdx.x;
	int max = 0;

	for (int i = block; i < min(256 + block, n); i++) {

		if (max < a[i]) {
			max = a[i];
		}
	}
	b[blockIdx.x] = max;
}

int main()
{

	int n;
	n = 3 >> 2;
	int a[n];

	for (int i = 0; i < n; i++) {
		a[i] = rand() % n;
		cout << a[i] << "\t";
	}

	cudaEvent_t start, end;
	int *ad, *bd;
	int size = n * sizeof(int);
	cudaMalloc(&ad, size);
	cudaMemcpy(ad, a, size, cudaMemcpyHostToDevice);
	int grids = ceil(n * 1.0f / 256.0f);
	cudaMalloc(&bd, grids * sizeof(int));

	dim3 grid(grids, 1);
	dim3 block(1, 1);

	cudaEventCreate(&start);
	cudaEventCreate(&end);
	cudaEventRecord(start);

	while (n > 1) {
		maxi<<<grids, block>>>(ad, bd, n);
		n = ceil(n * 1.0f / 256.0f);
		cudaMemcpy(ad, bd, n * sizeof(int), cudaMemcpyDeviceToDevice);
	}

	cudaEventRecord(end);
	cudaEventSynchronize(end);

	float time = 0;
	cudaEventElapsedTime(&time, start, end);

	int ans[2];
	cudaMemcpy(ans, ad, 4, cudaMemcpyDeviceToHost);

	cout << "The maximum element is : " << ans[0] << endl;

	cout << "The time required : ";
	cout << time << endl;
}


The maximum element is : 0
The time required : 0.004096



In [ ]:
%%cuda
#include <iostream>
#include <vector>
#include <cmath>

// CUDA kernel for parallel reduction to find the minimum value in an array
__global__ void minReduce(const int* arr, int* result, int N) {
    extern __shared__ int sharedData[];

    int tid = threadIdx.x;
    int i = blockIdx.x * blockDim.x + threadIdx.x;

    sharedData[tid] = (i < N) ? arr[i] : INT_MAX;
    __syncthreads();

    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s && i + s < N) {
            sharedData[tid] = min(sharedData[tid], sharedData[tid + s]);
        }
        __syncthreads();
    }

    if (tid == 0) {
        result[blockIdx.x] = sharedData[0];
    }
}

// CUDA kernel for parallel reduction to find the maximum value in an array
__global__ void maxReduce(const int* arr, int* result, int N) {
    extern __shared__ int sharedData[];

    int tid = threadIdx.x;
    int i = blockIdx.x * blockDim.x + threadIdx.x;

    sharedData[tid] = (i < N) ? arr[i] : INT_MIN;
    __syncthreads();

    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s && i + s < N) {
            sharedData[tid] = max(sharedData[tid], sharedData[tid + s]);
        }
        __syncthreads();
    }

    if (tid == 0) {
        result[blockIdx.x] = sharedData[0];
    }
}

// CUDA kernel for parallel reduction to find the sum of values in an array
__global__ void sumReduce(const int* arr, int* result, int N) {
    extern __shared__ int sharedData[];

    int tid = threadIdx.x;
    int i = blockIdx.x * blockDim.x + threadIdx.x;

    sharedData[tid] = (i < N) ? arr[i] : 0;
    __syncthreads();

    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s && i + s < N) {
            sharedData[tid] += sharedData[tid + s];
        }
        __syncthreads();
    }

    if (tid == 0) {
        result[blockIdx.x] = sharedData[0];
    }
}

int main() {
    const int N = 512; // Number of elements in the array
    const int blockSize = 256; // Threads per block

    // Generate random array
    std::vector<int> hostArr(N);
    for (int i = 0; i < N; ++i) {
        hostArr[i] = rand() % 1000; // Generate random numbers between 0 to 999
    }

    // Allocate device memory
    int *deviceArr, *deviceResult;
    cudaMalloc((void**)&deviceArr, N * sizeof(int));
    cudaMalloc((void**)&deviceResult, sizeof(int));

    // Copy data from host to device
    cudaMemcpy(deviceArr, hostArr.data(), N * sizeof(int), cudaMemcpyHostToDevice);

    // Compute grid size
    int gridSize = (N + blockSize - 1) / blockSize;

    // Find minimum
    minReduce<<<gridSize, blockSize, blockSize * sizeof(int)>>>(deviceArr, deviceResult, N);
    int minVal;
    cudaMemcpy(&minVal, deviceResult, sizeof(int), cudaMemcpyDeviceToHost);
    std::cout << "Minimum: " << minVal << std::endl;

    // Find maximum
    maxReduce<<<gridSize, blockSize, blockSize * sizeof(int)>>>(deviceArr, deviceResult, N);
    int maxVal;
    cudaMemcpy(&maxVal, deviceResult, sizeof(int), cudaMemcpyDeviceToHost);
    std::cout << "Maximum: " << maxVal << std::endl;

    // Find sum
    sumReduce<<<gridSize, blockSize, blockSize * sizeof(int)>>>(deviceArr, deviceResult, N);
    int sumVal;
    cudaMemcpy(&sumVal, deviceResult, sizeof(int), cudaMemcpyDeviceToHost);
    std::cout << "Sum: " << sumVal << std::endl;

    // Find average
    double average = static_cast<double>(sumVal) / N;
    std::cout << "Average: " << average << std::endl;

    // Free device memory
    cudaFree(deviceArr);
    cudaFree(deviceResult);

    return 0;
}


Minimum: 11
Maximum: 996
Sum: 131654
Average: 257.137



In [ ]:
%%cuda
#include <iostream>
#include <vector>
#include <cuda_runtime.h>

// Kernel function for vector addition
__global__ void vectorAdd(int* a, int* b, int* c, int n) {
    int index = threadIdx.x + blockIdx.x * blockDim.x;
    if (index < n) {
        c[index] = a[index] + b[index];
    }
}

int main() {
    const int N = 1000000; // Size of the vectors
    const int blockSize = 256; // Threads per block
    const int gridSize = (N + blockSize - 1) / blockSize; // Number of blocks

    // Initialize host vectors
    std::vector<int> hostA(N);
    std::vector<int> hostB(N);
    std::vector<int> hostC(N);

    // Fill host vectors with random values
    for (int i = 0; i < N; ++i) {
        hostA[i] = rand() % 1000;
        hostB[i] = rand() % 1000;
    }

    // Declare device pointers
    int *deviceA, *deviceB, *deviceC;

    // Allocate device memory
    cudaMalloc((void**)&deviceA, N * sizeof(int));
    cudaMalloc((void**)&deviceB, N * sizeof(int));
    cudaMalloc((void**)&deviceC, N * sizeof(int));

    // Copy host data to device
    cudaMemcpy(deviceA, hostA.data(), N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(deviceB, hostB.data(), N * sizeof(int), cudaMemcpyHostToDevice);

    // Launch kernel
    vectorAdd<<<gridSize, blockSize>>>(deviceA, deviceB, deviceC, N);

    // Copy result back to host
    cudaMemcpy(hostC.data(), deviceC, N * sizeof(int), cudaMemcpyDeviceToHost);

    // Verify results
    bool success = true;
    for (int i = 0; i < N; ++i) {
        if (hostC[i] != hostA[i] + hostB[i]) {
            success = false;
            break;
        }
    }

    if (success) {
        std::cout << "Vector addition successful!" << std::endl;
    } else {
        std::cout << "Vector addition failed!" << std::endl;
    }

    // Free device memory
    cudaFree(deviceA);
    cudaFree(deviceB);
    cudaFree(deviceC);

    return 0;
}


Vector addition successful!



In [ ]:
%%cuda
#include <iostream>
#include <vector>
#include <cuda_runtime.h>

// Kernel function for matrix multiplication
__global__ void matrixMul(const int* A, const int* B, int* C, int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < N && col < N) {
        int sum = 0;
        for (int k = 0; k < N; ++k) {
            sum += A[row * N + k] * B[k * N + col];
        }
        C[row * N + col] = sum;
    }
}

int main() {
    const int N = 1000; // Size of the matrices
    const int blockSize = 16; // Threads per block
    const int gridSize = (N + blockSize - 1) / blockSize; // Number of blocks

    // Initialize host matrices
    std::vector<int> hostA(N * N);
    std::vector<int> hostB(N * N);
    std::vector<int> hostC(N * N);

    // Fill host matrices with random values
    for (int i = 0; i < N * N; ++i) {
        hostA[i] = rand() % 10;
        hostB[i] = rand() % 10;
    }

    // Declare device pointers
    int *deviceA, *deviceB, *deviceC;

    // Allocate device memory
    cudaMalloc((void**)&deviceA, N * N * sizeof(int));
    cudaMalloc((void**)&deviceB, N * N * sizeof(int));
    cudaMalloc((void**)&deviceC, N * N * sizeof(int));

    // Copy host data to device
    cudaMemcpy(deviceA, hostA.data(), N * N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(deviceB, hostB.data(), N * N * sizeof(int), cudaMemcpyHostToDevice);

    // Launch kernel
    dim3 threadsPerBlock(blockSize, blockSize);
    dim3 numBlocks(gridSize, gridSize);
    matrixMul<<<numBlocks, threadsPerBlock>>>(deviceA, deviceB, deviceC, N);

    // Copy result back to host
    cudaMemcpy(hostC.data(), deviceC, N * N * sizeof(int), cudaMemcpyDeviceToHost);

    // Verify results
    bool success = true;
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            int sum = 0;
            for (int k = 0; k < N; ++k) {
                sum += hostA[i * N + k] * hostB[k * N + j];
            }
            if (hostC[i * N + j] != sum) {
                success = false;
                break;
            }
        }
        if (!success) {
            break;
        }
    }

    if (success) {
        std::cout << "Matrix multiplication successful!" << std::endl;
    } else {
        std::cout << "Matrix multiplication failed!" << std::endl;
    }

    // Free device memory
    cudaFree(deviceA);
    cudaFree(deviceB);
    cudaFree(deviceC);

    return 0;
}


Matrix multiplication successful!



In [ ]:
%%cuda
#include <iostream>
#include <vector>
#include <cuda_runtime.h>

// Kernel function for matrix multiplication
__global__ void matrixMul(const int* A, const int* B, int* C, int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < N && col < N) {
        int sum = 0;
        for (int k = 0; k < N; ++k) {
            sum += A[row * N + k] * B[k * N + col];
        }
        C[row * N + col] = sum;
    }
}

// Function to print a matrix
void printMatrix(const std::vector<int>& matrix, int N) {
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            std::cout << matrix[i * N + j] << " ";
        }
        std::cout << std::endl;
    }
}

int main() {
    const int N = 3; // Size of the matrices (for demonstration purposes)
    const int blockSize = 2; // Threads per block
    const int gridSize = (N + blockSize - 1) / blockSize; // Number of blocks

    // Initialize host matrices
    std::vector<int> hostA = {1, 2, 3, 4, 5, 6, 7, 8, 9};
    std::vector<int> hostB = {9, 8, 7, 6, 5, 4, 3, 2, 1};
    std::vector<int> hostC(N * N);

    // Print input matrices
    std::cout << "Matrix A:" << std::endl;
    printMatrix(hostA, N);
    std::cout << "Matrix B:" << std::endl;
    printMatrix(hostB, N);

    // Declare device pointers
    int *deviceA, *deviceB, *deviceC;

    // Allocate device memory
    cudaMalloc((void**)&deviceA, N * N * sizeof(int));
    cudaMalloc((void**)&deviceB, N * N * sizeof(int));
    cudaMalloc((void**)&deviceC, N * N * sizeof(int));

    // Copy host data to device
    cudaMemcpy(deviceA, hostA.data(), N * N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(deviceB, hostB.data(), N * N * sizeof(int), cudaMemcpyHostToDevice);

    // Launch kernel
    dim3 threadsPerBlock(blockSize, blockSize);
    dim3 numBlocks(gridSize, gridSize);
    matrixMul<<<numBlocks, threadsPerBlock>>>(deviceA, deviceB, deviceC, N);

    // Copy result back to host
    cudaMemcpy(hostC.data(), deviceC, N * N * sizeof(int), cudaMemcpyDeviceToHost);

    // Print result matrix
    std::cout << "Matrix C (A + B):" << std::endl;
    printMatrix(hostC, N);

    // Free device memory
    cudaFree(deviceA);
    cudaFree(deviceB);
    cudaFree(deviceC);

    return 0;
}


Matrix A:
1 2 3 
4 5 6 
7 8 9 
Matrix B:
9 8 7 
6 5 4 
3 2 1 
Matrix C (A + B):
30 24 18 
84 69 54 
138 114 90 



In [ ]:
%%cuda
#include <iostream>
#include <vector>
#include <cstdlib>
#include <ctime>

// CUDA kernel to add two vectors element-wise
__global__ void vectorAdd(const int *a, const int *b, int *c, int size) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < size) {
        c[tid] = a[tid] + b[tid];
    }
}

int main() {
    const int N = 1000; // Size of the vectors
    const int blockSize = 256; // Threads per block

    // Generate random vectors
    std::vector<int> hostA(N);
    std::vector<int> hostB(N);
    std::vector<int> hostC(N);

    srand(time(NULL));
    for (int i = 0; i < N; ++i) {
        hostA[i] = rand() % 1000; // Random numbers between 0 and 999
        hostB[i] = rand() % 1000;
    }

    // Allocate device memory
    int *deviceA, *deviceB, *deviceC;
    cudaMalloc((void**)&deviceA, N * sizeof(int));
    cudaMalloc((void**)&deviceB, N * sizeof(int));
    cudaMalloc((void**)&deviceC, N * sizeof(int));

    // Copy input vectors from host to device memory
    cudaMemcpy(deviceA, hostA.data(), N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(deviceB, hostB.data(), N * sizeof(int), cudaMemcpyHostToDevice);

    // Compute grid size
    int gridSize = (N + blockSize - 1) / blockSize;

    // Launch the kernel
    vectorAdd<<<gridSize, blockSize>>>(deviceA, deviceB, deviceC, N);

    // Copy result from device to host memory
    cudaMemcpy(hostC.data(), deviceC, N * sizeof(int), cudaMemcpyDeviceToHost);

    // Print original vectors and their sum vector
    std::cout << "Vector A: ";
    for (int i = 0; i < N; ++i) {
        std::cout << hostA[i] << " ";
    }
    std::cout << std::endl;

    std::cout << "Vector B: ";
    for (int i = 0; i < N; ++i) {
        std::cout << hostB[i] << " ";
    }
    std::cout << std::endl;

    std::cout << "Sum Vector: ";
    for (int i = 0; i < N; ++i) {
        std::cout << hostC[i] << " ";
    }
    std::cout << std::endl;

    // Free device memory
    cudaFree(deviceA);
    cudaFree(deviceB);
    cudaFree(deviceC);

    return 0;
}


Vector A: 697 474 491 133 578 201 683 133 400 101 845 581 702 693 894 959 406 817 871 609 702 313 888 649 408 365 684 540 856 946 138 342 339 442 166 363 565 606 826 93 710 840 757 485 613 715 337 648 922 943 845 309 983 216 876 396 120 37 616 194 306 684 671 24 478 34 273 86 963 770 963 482 754 664 42 303 120 344 24 737 687 530 818 921 270 325 561 669 327 37 544 70 32 807 135 262 667 136 760 620 805 616 753 34 970 801 468 530 287 260 586 597 667 92 495 810 9 18 63 546 180 252 572 500 824 985 634 17 20 81 525 519 669 353 646 469 247 474 856 382 86 443 303 22 625 895 698 389 656 803 641 129 346 764 282 587 867 90 195 197 920 675 859 690 158 818 470 744 608 723 8 611 693 397 518 764 391 760 855 117 648 869 588 287 826 438 673 61 455 958 109 115 514 750 30 439 155 797 626 643 447 991 836 839 567 969 225 176 312 813 145 373 265 748 959 48 178 879 147 657 221 742 63 889 548 70 660 699 318 169 191 36 312 735 338 526 371 709 539 514 6 485 732 591 984 680 681 249 336 805 81 938 951 591 950 634